In [ ]:
import io, os, sys, time
from zipfile import ZipFile
import zeep
import xmltodict, json, csv
import psycopg2
from configparser import ConfigParser
from datetime import datetime
import pytz
#import ctypes
#from ctypes.wintypes import MAX_PATH
import socket
from Indicadores import Indicadores
import html
from collections import OrderedDict

In [1]:
from Lattes import Lattes
lattes = Lattes()
lattes.id = '7281587998425548'

In [2]:
lattes.read_zip_from_disk()


Arquivo compactado lido no disco.


In [3]:
lattes.get_xml_from_zip()

Unzipping file and gettinf xml and json
retira, de cada JSON, caracteres especiais web
OrderedDict([('CURRICULO-VITAE', OrderedDict([('@SISTEMA-ORIGEM-XML', 'LATTES_OFFLINE'), ('@NUMERO-IDENTIFICADOR', '7281587998425548'), ('@DATA-ATUALIZACAO', '03022022'), ('@HORA-ATUALIZACAO', '113653'), ('DADOS-GERAIS', OrderedDict([('@NOME-COMPLETO', 'Alberto de Campos e Silva'), ('@NOME-EM-CITACOES-BIBLIOGRAFICAS', 'Campos Silva, A'), ('@NACIONALIDADE', 'B'), ('@PAIS-DE-NASCIMENTO', 'Brasil'), ('@UF-NASCIMENTO', 'GO'), ('@CIDADE-NASCIMENTO', 'Goiânia'), ('@PERMISSAO-DE-DIVULGACAO', 'NAO'), ('@DATA-FALECIMENTO', ''), ('@SIGLA-PAIS-NACIONALIDADE', 'BRA'), ('@PAIS-DE-NACIONALIDADE', 'Brasil'), ('@ORCID-ID', 'https://orcid.org/0000-0001-6506-1988'), ('RESUMO-CV', OrderedDict([('@TEXTO-RESUMO-CV-RH', 'Sou casado com uma mulher maravilhosa e tenho três lindos. Atualmente, sou Analista de C&T no CNPq, lotado na COCEX/CGCEX/DEHS, analisando principalmente processos relacionados à Física e à Matemática. Te

True

In [ ]:
print(lattes.json_lattes)

In [4]:
print(lattes.json_lattes['CURRICULO-VITAE']['DADOS-GERAIS']['RESUMO-CV']['@TEXTO-RESUMO-CV-RH'])

Sou casado com uma mulher maravilhosa e tenho três lindos. Atualmente, sou Analista de C&T no CNPq, lotado na COCEX/CGCEX/DEHS, analisando principalmente processos relacionados à Física e à Matemática. Tenho 46 anos, sou formado em física pela UnB, tendo dado aulas (de física) para o ensino fundamental por 2 anos (concursado GDF). Me mudei para Goiânia (por causa do baixo salário). Lá, trabalhei na justiça por 9 anos, aprendendo muito sobre direito. Como era encarregado inclusive da parte de informática, também aprendi programação de banco de dados (SQL), web-design e manutenção de micros. Um pouco ambicioso, comecei a me preparar para concursos públicos no intuito de melhorar minha qualidade de vida. Depois de três anos de estudos árduos, passei para o CNPq.
Fiz mestrado em Gestão Econômica do Meio Ambiente e agora estou inscrito em um programa de doutorado de educação em ciências. Apesar dos títulos (em economia e em educação), ambos os trabalhos são, na verdade, o uso da cientometri

In [ ]:
def myprint(d, path=None):
    if not path: path = []
    for k, v in d.items():
        if isinstance(v, dict) or isinstance(v, OrderedDict):
            #print("Getting: ", k)
            myprint(v, path + [k])
        else:
            if isinstance(v, list):
                num = 0
                for item in v:
                    #print("Unfolding List: ", v)
                    myprint(item, path + [k] + [num])
                    num +=1
            else:
                if not v == None:
                    d[k] = html.unescape(v)
                    print("{2} - {0} : {1}".format(k, html.unescape(v), path))
                else:
                    print("{2} - {0} : {1}".format(k, v, path))

In [ ]:
    def html_unescape(variável, valor):
        variável = html.unescape(valor)
    
    def recorre_sobre_todo_json(d, path=None, funcion = html_unescape):
        if not path: path = []
        for k, v in d.items():
            if isinstance(v, dict) or isinstance(v, OrderedDict):
                #print("Getting: ", k)
                recorre_sobre_todo_json(v, path + [k])
            elif isinstance(v, list):
                num = 0
                for item in v:
                    #print("Unfolding List: ", v)
                    recorre_sobre_todo_json(item, path + [k] + [num])
                    num +=1
            else:
                if not v == None:
                    funcion(d[k], v)
                    print("{2} - {0} : {1}".format(k, html.unescape(v), path))
                else:
                    print("{2} - {0} : {1}".format(k, v, path))
                    pass
        return d

In [ ]:
print(recorre_sobre_todo_json(lattes.json_lattes))

In [ ]:
def listRecursive (d, key, path = None):
    if not path: path = []
    for k, v in d.items ():
        if isinstance (v, OrderedDict) or isinstance(v, dict):
            for path, found in listRecursive (v, key, path + [k] ):
                yield path, found
        if k == key:
            yield path + [k], v

for path, found in listRecursive (json_lattes, 'AUTORES'):
    print (path, found)

In [ ]:
print(lattes.json_lattes)

In [ ]:
json_lattes = xmltodict.parse(lattes.xml)